In [3]:
import dynesty
import numpy
from scipy import stats

#likelihood function.


def loglikelihood(x):

    #2 uncorrelated gaussians with mean : 1.0 and 6.0 and standard deviations: 1.0 and 0.5
    L = (-0.5*(((x[6]-1.0)/1.0)**2)
         - 0.5*(((x[4]-6.0)/0.5)**2)
         - numpy.log(1.0*numpy.sqrt(2*numpy.pi))
         - numpy.log(0.5*numpy.sqrt(2*numpy.pi))
         )

    return L

#prior transforms


def prior_transform(u):

    x = numpy.array(u)

    for i, s in enumerate(sampling_parameters):
        if s[-1] == 'Normal':
            mean = s[1]
            sigma = s[2]
            x[i] = scipy.stats.norm.ppf(u[i], loc=mean, scale=sigma)
        elif s[-1] == 'Turncated_Normal':
            mean = s[1]
            sigma = s[2]
            low = (s[3]-mean)/sigma
            high = (s[4]-mean)/sigma
            x[i] = scipy.stats.truncnorm.ppf(
                u[i], low, high, loc=mean, scale=sigma)
        elif s[-1] == 'Uniform':
            x[i] = (s[2]-s[1])*u[i] + s[1]

    return x


#a tuple of parameters to sample from
sampling_parameters = [('Porb', 8.46, 0.0000230, 'Normal'),
                       ('feh', -1.06, 0.3, -1.014, 0.537, 'Truncated_normal'),
                       ('eccentricity', 0.042, 0.01,
                        0.0, 0.45, 'Truncated_normal'),
                       ('Wdisk', 2*scipy.pi/14, 2*scipy.pi/1.4, 'Uniform'),
                       ('logQ', 5.0, 12.0, 'Uniform'),
                       ('primary_mass', 0.5, 1.2, 'Uniform'),
                       ('age', 1e-3, 10.0, 'Uniform')]

ndim = len(sampling_parameters)

dsampler = dynesty.NestedSampler(loglikelihood, prior_transform,
                                 ndim, nlive=1500)

dsampler.run_nested()
dresults = dsampler.results
dresults.summary()


4954it [00:02, 1994.45it/s, +1500 | bound: 0 | nc: 1 | ncall: 39420 | eff(%): 16.372 | loglstar:   -inf < -1.146 <    inf | logz: -4.500 +/-  0.041 | dlogz:  0.001 >  1.509]

Summary
nlive: 1500
niter: 4954
ncall: 39420
eff(%): 16.372
logz: -4.500 +/-  0.041
